In [1]:
%cd ..
%reload_ext autoreload
%autoreload 2

D:\ownCloud\Uni\Semester Ma 5\3D Scanning & Motion Capture (IN2354)\Project\3D-FaceReconstruction


In [4]:
from face_reconstruction.landmarks import load_landmarks
from face_reconstruction.model import BaselFaceModel

In [3]:
landmarks = load_landmarks("model2019_face12_landmarks")

In [6]:
bfm = BaselFaceModel.from_h5("model2019_face12.h5")

In [7]:
landmarks

{1: 12550,
 2: 25563,
 3: 11735,
 4: 107,
 5: 2926,
 6: 585,
 7: 26043,
 8: 23314,
 9: 429,
 10: 5494,
 11: 4710,
 12: 361,
 13: 11847,
 14: 18345,
 15: 20967,
 16: 14965,
 17: 26053,
 18: 22083,
 19: 10487,
 20: 27365,
 21: 5909,
 22: 24076,
 23: 23596,
 24: 22244,
 25: 24263,
 26: 24301,
 27: 17422,
 28: 26082,
 29: 24204,
 30: 4580,
 31: 45,
 32: 27273,
 33: 1081,
 34: 24839,
 35: 21702,
 36: 12561,
 37: 6898,
 38: 6740,
 39: 14119,
 40: 16732,
 41: 23309,
 42: 24290,
 43: 20843,
 44: 26598,
 45: 14406,
 46: 5534,
 47: 11246,
 48: 19783,
 49: 11660,
 50: 12568,
 51: 25963,
 52: 1104,
 53: 7951,
 54: 12703,
 55: 4945,
 56: 24720,
 57: 7451,
 58: 22167,
 59: 6942,
 60: 25919,
 61: 15130,
 62: 14765,
 63: 19732,
 64: 2225,
 65: 9718,
 66: 17791,
 67: 13689,
 68: 4140}